In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


22/11/06 16:59:56 WARN Utils: Your hostname, DESKTOP-TVDS9T6 resolves to a loopback address: 127.0.1.1; using 172.29.38.221 instead (on interface eth0)
22/11/06 16:59:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/06 16:59:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pandas as pd
import dask.dataframe as dd

X = pd.DataFrame(zip(range(100), range(100)), columns=['x','y'])
X_dask = dd.from_pandas(X, npartitions=1)
X_spark = spark.createDataFrame(X)

/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [20]:
from dfender import distributable

class Test:
    @distributable(pd.Series)
    def predict(self, X: pd.Series) -> pd.Series:
        return X - 2
        # return (pd.to_datetime('2000-01-01').to_period('M') + X).dt.to_timestamp()

    @distributable((pd.Series, pd.DataFrame))
    def rowmax(self, X: pd.DataFrame):
        return X.max(axis=1)


est = Test()
est.predict(X['x'])
est.predict(X_dask['x']).compute()

X_spark.select([
    "x",
    est.predict(X_spark.select(['x'])).alias('pred')
]).show()

/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+---+----+
|  x|pred|
+---+----+
|  0|{-2}|
|  1|{-1}|
|  2| {0}|
|  3| {1}|
|  4| {2}|
|  5| {3}|
|  6| {4}|
|  7| {5}|
|  8| {6}|
|  9| {7}|
| 10| {8}|
| 11| {9}|
| 12|{10}|
| 13|{11}|
| 14|{12}|
| 15|{13}|
| 16|{14}|
| 17|{15}|
| 18|{16}|
| 19|{17}|
+---+----+
only showing top 20 rows



In [18]:
X_spark.select([
    "x",
    est.predict(X_spark.select(['x'])).alias('pred')
]).collect()

/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


[Row(x=0, pred=Row(x=-2)),
 Row(x=1, pred=Row(x=-1)),
 Row(x=2, pred=Row(x=0)),
 Row(x=3, pred=Row(x=1)),
 Row(x=4, pred=Row(x=2)),
 Row(x=5, pred=Row(x=3)),
 Row(x=6, pred=Row(x=4)),
 Row(x=7, pred=Row(x=5)),
 Row(x=8, pred=Row(x=6)),
 Row(x=9, pred=Row(x=7)),
 Row(x=10, pred=Row(x=8)),
 Row(x=11, pred=Row(x=9)),
 Row(x=12, pred=Row(x=10)),
 Row(x=13, pred=Row(x=11)),
 Row(x=14, pred=Row(x=12)),
 Row(x=15, pred=Row(x=13)),
 Row(x=16, pred=Row(x=14)),
 Row(x=17, pred=Row(x=15)),
 Row(x=18, pred=Row(x=16)),
 Row(x=19, pred=Row(x=17)),
 Row(x=20, pred=Row(x=18)),
 Row(x=21, pred=Row(x=19)),
 Row(x=22, pred=Row(x=20)),
 Row(x=23, pred=Row(x=21)),
 Row(x=24, pred=Row(x=22)),
 Row(x=25, pred=Row(x=23)),
 Row(x=26, pred=Row(x=24)),
 Row(x=27, pred=Row(x=25)),
 Row(x=28, pred=Row(x=26)),
 Row(x=29, pred=Row(x=27)),
 Row(x=30, pred=Row(x=28)),
 Row(x=31, pred=Row(x=29)),
 Row(x=32, pred=Row(x=30)),
 Row(x=33, pred=Row(x=31)),
 Row(x=34, pred=Row(x=32)),
 Row(x=35, pred=Row(x=33)),
 Row(x=36, p

In [5]:
from pyspark.sql.functions import struct, pandas_udf

@pandas_udf("long")
def test(df):
    return df.prod(axis=1)

X_spark.select(test(struct(*X_spark.columns)).alias('new')).collect()

[Row(new=0),
 Row(new=1),
 Row(new=4),
 Row(new=9),
 Row(new=16),
 Row(new=25),
 Row(new=36),
 Row(new=49),
 Row(new=64),
 Row(new=81),
 Row(new=100),
 Row(new=121),
 Row(new=144),
 Row(new=169),
 Row(new=196),
 Row(new=225),
 Row(new=256),
 Row(new=289),
 Row(new=324),
 Row(new=361),
 Row(new=400),
 Row(new=441),
 Row(new=484),
 Row(new=529),
 Row(new=576),
 Row(new=625),
 Row(new=676),
 Row(new=729),
 Row(new=784),
 Row(new=841),
 Row(new=900),
 Row(new=961),
 Row(new=1024),
 Row(new=1089),
 Row(new=1156),
 Row(new=1225),
 Row(new=1296),
 Row(new=1369),
 Row(new=1444),
 Row(new=1521),
 Row(new=1600),
 Row(new=1681),
 Row(new=1764),
 Row(new=1849),
 Row(new=1936),
 Row(new=2025),
 Row(new=2116),
 Row(new=2209),
 Row(new=2304),
 Row(new=2401),
 Row(new=2500),
 Row(new=2601),
 Row(new=2704),
 Row(new=2809),
 Row(new=2916),
 Row(new=3025),
 Row(new=3136),
 Row(new=3249),
 Row(new=3364),
 Row(new=3481),
 Row(new=3600),
 Row(new=3721),
 Row(new=3844),
 Row(new=3969),
 Row(new=4096),
 Row(n

In [6]:
est.predict(X_spark.select(['x']))

/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Column<'func_(struct(x, x))'>

In [7]:
from pyspark.sql.functions import struct

struct(*X_spark.columns)

Column<'struct(x, y)'>

In [8]:
from pyspark.sql.functions import pandas_udf

@pandas_udf("bigint")
def test(df):
    return df - 2

X_spark.select(test("x")).collect()

[Row(test(x)=-2),
 Row(test(x)=-1),
 Row(test(x)=0),
 Row(test(x)=1),
 Row(test(x)=2),
 Row(test(x)=3),
 Row(test(x)=4),
 Row(test(x)=5),
 Row(test(x)=6),
 Row(test(x)=7),
 Row(test(x)=8),
 Row(test(x)=9),
 Row(test(x)=10),
 Row(test(x)=11),
 Row(test(x)=12),
 Row(test(x)=13),
 Row(test(x)=14),
 Row(test(x)=15),
 Row(test(x)=16),
 Row(test(x)=17),
 Row(test(x)=18),
 Row(test(x)=19),
 Row(test(x)=20),
 Row(test(x)=21),
 Row(test(x)=22),
 Row(test(x)=23),
 Row(test(x)=24),
 Row(test(x)=25),
 Row(test(x)=26),
 Row(test(x)=27),
 Row(test(x)=28),
 Row(test(x)=29),
 Row(test(x)=30),
 Row(test(x)=31),
 Row(test(x)=32),
 Row(test(x)=33),
 Row(test(x)=34),
 Row(test(x)=35),
 Row(test(x)=36),
 Row(test(x)=37),
 Row(test(x)=38),
 Row(test(x)=39),
 Row(test(x)=40),
 Row(test(x)=41),
 Row(test(x)=42),
 Row(test(x)=43),
 Row(test(x)=44),
 Row(test(x)=45),
 Row(test(x)=46),
 Row(test(x)=47),
 Row(test(x)=48),
 Row(test(x)=49),
 Row(test(x)=50),
 Row(test(x)=51),
 Row(test(x)=52),
 Row(test(x)=53),
 R

In [9]:
est.predict(X_spark.select(['x'])).collect()

/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


TypeError: 'Column' object is not callable

In [ ]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import LongType
@pandas_udf("long")
def test(df: pd.Series) -> pd.Series:
    return (df / 2)

X_spark.select([test("x").alias('y')]).collect()

[Row(y=0),
 Row(y=0),
 Row(y=1),
 Row(y=1),
 Row(y=2),
 Row(y=2),
 Row(y=3),
 Row(y=3),
 Row(y=4),
 Row(y=4),
 Row(y=5),
 Row(y=5),
 Row(y=6),
 Row(y=6),
 Row(y=7),
 Row(y=7),
 Row(y=8),
 Row(y=8),
 Row(y=9),
 Row(y=9),
 Row(y=10),
 Row(y=10),
 Row(y=11),
 Row(y=11),
 Row(y=12),
 Row(y=12),
 Row(y=13),
 Row(y=13),
 Row(y=14),
 Row(y=14),
 Row(y=15),
 Row(y=15),
 Row(y=16),
 Row(y=16),
 Row(y=17),
 Row(y=17),
 Row(y=18),
 Row(y=18),
 Row(y=19),
 Row(y=19),
 Row(y=20),
 Row(y=20),
 Row(y=21),
 Row(y=21),
 Row(y=22),
 Row(y=22),
 Row(y=23),
 Row(y=23),
 Row(y=24),
 Row(y=24),
 Row(y=25),
 Row(y=25),
 Row(y=26),
 Row(y=26),
 Row(y=27),
 Row(y=27),
 Row(y=28),
 Row(y=28),
 Row(y=29),
 Row(y=29),
 Row(y=30),
 Row(y=30),
 Row(y=31),
 Row(y=31),
 Row(y=32),
 Row(y=32),
 Row(y=33),
 Row(y=33),
 Row(y=34),
 Row(y=34),
 Row(y=35),
 Row(y=35),
 Row(y=36),
 Row(y=36),
 Row(y=37),
 Row(y=37),
 Row(y=38),
 Row(y=38),
 Row(y=39),
 Row(y=39),
 Row(y=40),
 Row(y=40),
 Row(y=41),
 Row(y=41),
 Row(y=42),


In [ ]:
est.predict(X_spark.select(['x'])).collect()

/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


x bigint
22/11/06 16:35:40 ERROR Executor: Exception in task 16.0 in stage 1.0 (TID 40)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_28773/557874211.py", line 6, in predict
TypeError: unsupported operand type(s) for -: 'map' and 'int'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:559)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:101)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:50)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPla

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_28773/557874211.py", line 6, in predict
TypeError: unsupported operand type(s) for -: 'map' and 'int'
